In [153]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [154]:
import pandas as pd
import re
from collections import Counter
import numpy as np
import xml.etree.ElementTree as ET
import json
import os

In [155]:
# Load your data
df = pd.read_excel(r'C:\Users\vmd1\OneDrive - NIST\Desktop\Programming\Entire CDD.xlsx', usecols=['Data Element Name', 'Definition', 'revised definition', 'Data Type', 'revised data type'], dtype={'Data Element Name': str})

In [156]:
# STOP WORD LIBRARY BELOW








In [157]:
# Check for revised definitions and data types
for i in range(len(df)):
    
    cdd_definition = df.at[i, 'Definition']
    cdd_revised_definition = df.at[i, 'revised definition']

    if not pd.isna(cdd_revised_definition) and cdd_revised_definition.strip() != '':
        cdd_definition = cdd_revised_definition

    df.at[i, 'Definition'] = cdd_definition

    cdd_data_type = df.at[i, 'Data Type']
    cdd_revised_data_type = df.at[i, 'revised data type']

    if not pd.isna(cdd_revised_data_type) and cdd_revised_data_type.strip() != '':
        cdd_data_type = cdd_revised_data_type

    df.at[i, 'Data Type'] = cdd_data_type

if 'Data Element Name (Processed)' not in df.columns:
    df['Data Element Name (Processed)'] = ""

# print(df.loc[141:164])

In [193]:
# Stop-word list
words_to_filter = []

# Function to count words and update the stop-word list
def count_words():
    # Combine all entries in the 'Data Element Name' column into a single string
    text = ' '.join(df['Data Element Name'].dropna())
    
    # Use regex to split the text into words (ignoring punctuation and making it case-insensitive)
    words = re.findall(r'\b\w+\b', text.lower())
    
    # Count the frequency of each word
    word_freq = Counter(words)
    
    # Calculate the total number of words
    total_words = sum(word_freq.values())
    
    # Sort the word frequencies from greatest to least
    sorted_word_freq = word_freq.most_common()
    
    # Print the frequencies and percentages
    print(f'Total words: {total_words}')
    print(f'{"Word":<20}{"Count":<10}{"Percentage":<10}')
    print('-' * 40)
    for word, freq in sorted_word_freq:
        percentage = (freq / total_words) * 100
        print(f'{word:<20}{freq:<10}{percentage:.2f}%')
        
        # Adding words with counts over 15 to the stop-word list
        if percentage > 2.00:
            words_to_filter.append(word.upper())

    # print("\nStop Words:", words_to_filter)

# Call the function
count_words()

Total words: 1354
Word                Count     Percentage
----------------------------------------
nan                 67        4.95%
id                  50        3.69%
build               48        3.55%
am                  44        3.25%
type                41        3.03%
process             38        2.81%
material            32        2.36%
tic                 32        2.36%
machine             30        2.22%
part                27        1.99%
system              25        1.85%
platform            22        1.62%
calibration         21        1.55%
date                20        1.48%
gas                 19        1.40%
equipment           18        1.33%
specimen            17        1.26%
enumeration         16        1.18%
project             16        1.18%
monitoring          15        1.11%
location            14        1.03%
name                13        0.96%
software            13        0.96%
stock               13        0.96%
list                13        0.96%


In [194]:
print("\nStop Words:", words_to_filter)


Stop Words: ['NAN', 'ID', 'BUILD', 'AM', 'TYPE', 'PROCESS', 'MATERIAL', 'TIC', 'MACHINE']


In [195]:
# Function to process each data element name
def process_data_element_name(name):
    # Handle missing or non-string values
    if isinstance(name, float):
        return ""
    
    # Split the name into words
    words = name.split()
    
    # Filter out specified words
    filtered_words = [word for word in words if word.upper() not in words_to_filter]
    
    # Join the remaining words without spaces
    processed_name = ''.join(filtered_words)
    
    return processed_name

# Apply the processing function to each data element name
df['Data Element Name (Processed)'] = df['Data Element Name'].apply(process_data_element_name)

print(df.loc[141:164])

                Data Element Name  \
141                      Build ID   
142              Software ID list   
143               Organization ID   
144            NonAMEquip ID list   
145              Material ID list   
146               Build Personnel   
147             SynthData ID list   
148                    Build Plan   
149        Build Plate Definition   
150                    Build Type   
151                    Project ID   
152             Build Platform ID   
153             AM System ID list   
154            Built Part ID list   
155               Process ID list   
156              Specimen ID list   
157          Process Data ID list   
158                   TIC ID list   
159            Simulation ID list   
160           Part Design ID list   
161                Build Platform   
162                Build Platform   
163                Build Job File   
164  Build Platform Configuration   

                                            Definition  \
141  dentifier f

In [196]:
# count duplicates prior to replacing empty strings and duplicates with original name
# Get the 'Data Element Name (Processed)' column
processed_names = df['Data Element Name (Processed)']

# Detect duplicated element names
duplicates = processed_names[processed_names.duplicated(keep=False)]

# Detect empty strings
empty_strings = processed_names[processed_names == ""]

# Print duplicates with their respective locations and count the number of instances of duplication
print("Duplicates:")
duplicate_count = 0
if duplicates.empty:
    print("No duplicates found.")
else:
    for name in duplicates.unique():
        duplicate_rows = duplicates[duplicates == name].index.tolist()
        instance_count = len(duplicate_rows)
        duplicate_count += instance_count
        print(f'"{name}" is duplicated in rows: {duplicate_rows}')
    print(f"Total number of duplicated words: {duplicates.nunique()}")
    print(f"Total number of instances of duplication: {duplicate_count}")

# Print empty strings with their respective locations
print("\nEmpty Strings:")
if empty_strings.empty:
    print("No empty strings found.")
else:
    for index in empty_strings.index:
        print(f'Row {index}: Empty string.')
    print(f"Total number of empty strings: {empty_strings.count()}")


Duplicates:
"-" is duplicated in rows: [0, 140, 165, 174, 192, 244, 254, 267, 308, 357, 373, 389, 400, 454, 461]
"" is duplicated in rows: [1, 28, 29, 47, 58, 61, 83, 85, 86, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 124, 137, 141, 150, 177, 193, 231, 261, 269, 270, 277, 280, 298, 309, 320, 326, 336, 337, 338, 339, 340, 344, 398, 401, 402, 413, 421, 427, 440, 443, 444, 446, 469, 470, 471, 472, 473, 474]
"System" is duplicated in rows: [2, 4, 87]
"Name" is duplicated in rows: [16, 194, 412]
"Manufacturer" is duplicated in rows: [17, 211]
"Model" is duplicated in rows: [18, 92]
"SerialNumber" is duplicated in rows: [19, 93]
"Location" is duplicated in rows: [90, 312, 419]
"Organization" is duplicated in rows: [91, 143, 246, 247]
"CalibrationOperator" is duplicated in rows: [125, 126]
"Calibration" is duplicated in rows: [127, 128]
"CalibrationResult" is duplicated in rows: [131, 132]
"CalibrationNotes" is 

In [197]:
# Removes all spaces between words and capitalizes first letter of each word
def remove_spaces(input_string):
    # Split the string into words based on spaces
    words = input_string.split()
    # Capitalize the first letter of each word, keeping fully capitalized words as they are
    capitalized_words = [word if word.isupper() else word.capitalize() for word in words]
    # Join the words without spaces
    no_space_string = ''.join(capitalized_words)
    return no_space_string

In [198]:
# Make sure everything is a string
df = df.astype(str)

# Create a DataFrame with the required columns
processed_element_names_df = df[['Data Element Name', 'Data Element Name (Processed)', 'Definition', 'revised definition', 'Data Type', 'revised data type']]

# Process data element name to end up with final data element name
for index, row in processed_element_names_df.iterrows():
    data_element_name = row['Data Element Name']
    processed_data_element_name = row['Data Element Name (Processed)']
    definition = row['Definition']
    data_type = row['Data Type']

    # Check for empty strings
    if processed_data_element_name == 'nan' or processed_data_element_name == '':
        processed_data_element_name = data_element_name
        processed_element_names_df.at[index, 'Data Element Name (Processed)'] = data_element_name
    # Check for duplicates
    else:
        duplicate_check = processed_element_names_df['Data Element Name (Processed)'] == processed_data_element_name
        if duplicate_check.sum() > 1:
            processed_data_element_name = data_element_name
            processed_element_names_df.at[index, 'Data Element Name (Processed)'] = data_element_name

    processed_element_names_df.at[index, 'Data Element Name (Processed)'] = remove_spaces(processed_data_element_name)
    
# Print the DataFrame to check the results
print(processed_element_names_df.loc[141:164])

                Data Element Name Data Element Name (Processed)  \
141                      Build ID                       BuildID   
142              Software ID list                  Softwarelist   
143               Organization ID                OrganizationID   
144            NonAMEquip ID list                Nonamequiplist   
145              Material ID list                MaterialIDList   
146               Build Personnel                BuildPersonnel   
147             SynthData ID list                 Synthdatalist   
148                    Build Plan                          Plan   
149        Build Plate Definition               Platedefinition   
150                    Build Type                     BuildType   
151                    Project ID                     ProjectID   
152             Build Platform ID               BuildPlatformID   
153             AM System ID list                    Systemlist   
154            Built Part ID list                 Builtpartlis

In [199]:
# IMPLEMENTING STOP WORD LIBRARY INTO CDM BELOW







In [200]:
# Function to convert string into camel case except for ID
def to_camel_case(snake_str):
    # Split the string by spaces or underscores
    components = snake_str.replace("_", " ").split()
    # Handle the first component (lowercase)
    camel_case_string = components[0].lower()
    # Process remaining components
    for component in components[1:]:
        if component.upper() == 'ID':
            camel_case_string += 'ID'
        else:
            camel_case_string += component.title()
    return camel_case_string

In [201]:
# Import processed CDD as a dataframe
proc_CDD_df = proc_CDD_df = processed_element_names_df

# Import processed CDM as a dataframe
proc_CDM = r'C:\Users\vmd1\OneDrive - NIST\Desktop\Programming\build bucket files\Build Only - CDM Processed.xlsx'
proc_CDM_df = pd.read_excel(proc_CDM, usecols=['Data Element Name (OG)', 'Data Element Name (Processed)', 'Definition', 'Cardinality (1 or n)', 'Data Type', 'Alias Name'])

# Ensure data types are consistent
dtype = {
    'Data Element Name (OG)': str,
    'Data Element Name (Processed)': str,
    'Definition': str,
    'Cardinality (1 or n)': str,
    'Data Type': str,
    'Alias Name': str
}

proc_CDM_df = proc_CDM_df.astype(dtype)

print(proc_CDD_df.loc[141:164])
print(proc_CDM_df)

                Data Element Name Data Element Name (Processed)  \
141                      Build ID                       BuildID   
142              Software ID list                  Softwarelist   
143               Organization ID                OrganizationID   
144            NonAMEquip ID list                Nonamequiplist   
145              Material ID list                MaterialIDList   
146               Build Personnel                BuildPersonnel   
147             SynthData ID list                 Synthdatalist   
148                    Build Plan                          Plan   
149        Build Plate Definition               Platedefinition   
150                    Build Type                     BuildType   
151                    Project ID                     ProjectID   
152             Build Platform ID               BuildPlatformID   
153             AM System ID list                    Systemlist   
154            Built Part ID list                 Builtpartlis

In [202]:
def match(CDM_df, CDD_df):
    matched_elements = []

    # Iterate through each element from CDM
    for u in range(len(CDM_df)):
        cdm_proc_element_name = CDM_df.at[u, 'Data Element Name (Processed)']
        cdm_element_type = CDM_df.at[u, 'Data Type']
        cdm_element_def = CDM_df.at[u, 'Definition']

        # Create a flag to check if a match is found
        match_found = False
        
        # Iterate through each element from CDD
        for i in range(len(CDD_df)):
            cdd_proc_element_name = CDD_df.at[i, 'Data Element Name (Processed)']
            cdd_definition = CDD_df.at[i, 'Definition']
            cdd_data_type = CDD_df.at[i, 'Data Type']
            cdd_element_name = CDD_df.at[i, 'Data Element Name']
            cdd_element_name = to_camel_case(cdd_element_name)

            # CASE 1: THE NAMES ARE THE SAME
            if cdd_proc_element_name == cdm_proc_element_name:
                matched_element = {
                    'Element Name': cdd_element_name,
                    'Type': cdd_data_type,
                    'Documentation': cdd_definition,
                }
                matched_elements.append(matched_element)
                match_found = True
                print("match found")
                break  # Assuming one match per element is sufficient

        # If no match was found, add the element with 'No Match' status
        if not match_found:
            unmatched_element = {
                'Element Name': CDM_df.at[u, 'Data Element Name (OG)'],
                'Type': CDM_df.at[u, 'Data Type'],
                'Documentation': CDM_df.at[u, 'Definition'],
            }
            print("no match") # for ease of counting how many matches were not successfully made
            matched_elements.append(unmatched_element)

    return matched_elements

# Function to extract information from CDM file
def extract_element_details(elem, df):
    details = {}
    details['Element Name'] = elem.get('name')
    details['Type'] = elem.get('type')
    details['Min Occurs'] = elem.get('minOccurs')
    details['Max Occurs'] = elem.get('maxOccurs')
    
    doc_elem = elem.find(".//{http://www.w3.org/2001/XMLSchema}documentation")
    if doc_elem is not None and doc_elem.text:
        details['Documentation'] = doc_elem.text.strip()
        match(details, df)
    
    return details

# Define the path to the XSD file
file_path = r"C:\Users\vmd1\OneDrive - NIST\Documents\preeditbuild.xsd"


# Usage example
matches = match(proc_CDM_df, proc_CDD_df)
json_data = json.dumps(matches, indent=2)
print(json_data)

match found
no match
match found
no match
no match
no match
match found
no match
match found
match found
match found
no match
no match
match found
no match
match found
match found
no match
[
  {
    "Element Name": "buildID",
    "Type": "string",
    "Documentation": "dentifier for an AM build; This field is required to be unique across all instances of data records"
  },
  {
    "Element Name": "buildType",
    "Type": "string",
    "Documentation": "Description of the primary purpose of the Build (end use component, specimens, research, etc.)"
  },
  {
    "Element Name": "simulationID",
    "Type": "string",
    "Documentation": "Unique identifier for a specific simulation"
  },
  {
    "Element Name": "feedstockMaterial",
    "Type": "Material",
    "Documentation": "Material consumed during build"
  },
  {
    "Element Name": "quantityOfFeedstockMaterialUsed",
    "Type": "Measurement",
    "Documentation": "Mass of material used in solidified parts"
  },
  {
    "Element Name": 

In [203]:
# Get the 'Data Element Name (Processed)' column
processed_names = processed_element_names_df['Data Element Name (Processed)']

# Detect duplicated element names
duplicates = processed_names[processed_names.duplicated(keep=False)]

# Detect empty strings
empty_strings = processed_names[processed_names == ""]

# Print duplicates with their respective locations
print("Duplicates:")
if duplicates.empty:
    print("No duplicates found.")
else:
    for name in duplicates.unique():
        duplicate_rows = duplicates[duplicates == name].index.tolist()
        print(f'"{name}" is duplicated in rows: {duplicate_rows}')

# Print empty strings with their respective locations
print("\nEmpty Strings:")
if empty_strings.empty:
    print("No empty strings found.")
else:
    for index in empty_strings.index:
        print(f'Row {index}: Empty string.')


Duplicates:
"-" is duplicated in rows: [0, 140, 165, 174, 192, 244, 254, 267, 308, 357, 373, 389, 400, 454, 461]
"Nan" is duplicated in rows: [1, 28, 29, 47, 58, 61, 83, 85, 86, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 124, 137, 177, 193, 231, 261, 269, 277, 280, 298, 309, 320, 326, 336, 337, 338, 339, 340, 344, 398, 401, 421, 427, 440, 443, 444, 446, 469, 470, 471, 472, 473, 474]
"OrganizationID" is duplicated in rows: [143, 246]
"BuildPlatformID" is duplicated in rows: [152, 232]
"BuildPlatform" is duplicated in rows: [161, 162]

Empty Strings:
No empty strings found.
